In [5]:
import pickle
import time
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import connectivity_measures as cm

In [6]:
### Contructing filters
### Butterworth filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return b, a

fs      = 1000.0

filter_parameters = {'theta':    {'N': 4, 'lowcut': 3.5,  'highcut': 7.5},
                    'alpha':     {'N': 4, 'lowcut': 8.0,  'highcut': 13.0},
                    'beta':      {'N': 4, 'lowcut': 13.0, 'highcut': 25.0},
                    'low_gamma': {'N': 6, 'lowcut': 25.0, 'highcut': 60}}

butterworths = dict()
for band in filter_parameters.keys():
    lowcut = filter_parameters[band]['lowcut']
    highcut = filter_parameters[band]['highcut']
    N = filter_parameters[band]['N']
    butterworths[band] = butter_bandpass(lowcut, highcut, fs, order=N)


In [7]:
IDs = ['SERT1597']

In [11]:
condition = 'epochs'

iCohs = {}
for mouse in IDs: 
    npys_dir = '/home/maspe/filer/SERT/' + mouse + '/npys/'
    print('\nLoading mouse {}...'.format(mouse))
    
    ### Loading data
    data = pickle.load(open(npys_dir + mouse + '.epochs', 'rb'))
    
    ### Loop
    filtered_bands = {'theta': {}, 'low_gamma': {}}
    iterator = 0
    for structure in data.keys(): 
        if iterator == 0:
            all_structures = data[structure][condition]
        else:
            all_structures = np.vstack((all_structures, data[structure][condition]))
        
        iterator += 1
        
    print('Filtering...')    
    for band in filtered_bands.keys():
        filtered = signal.filtfilt(b=butterworths[band][0], a=butterworths[band][1],
                                   x=all_structures, axis=1)
        
        print('Calculating iCoh for {} band...'.format(band))
        clock = time.time()

        n_epochs = filtered.shape[2]
        for epoch in range(n_epochs):
            if epoch == 0:
                icoh = cm.icoh(filtered[:,:,epoch], average = False)
            else:
                icoh = np.dstack((icoh, cm.icoh(filtered[:,:,epoch], average = False)))
        
        print('iCoh calculated in {} min'.format((time.time() - clock)) / 60)
        
        iCohs[band] = icoh

    
    pickle.dump(iCohs, open(npys_dir + '.icoh', 'wb'), protocol=2)

    
print('Done!')      


Loading mouse SERT1597...
Filtering...
Calculating iCoh...
iCoh calculated in 245.968783855 s
Calculating iCoh...
iCoh calculated in 242.087009907 s
Done!


In [13]:
icoh.shape

(32, 32, 16)

In [ ]:
band = 'theta'
structure = 'mPFC'

icoh  = cm.icoh(filtered_bands[band][structure], average = False)


In [ ]:
filtered_bands[band][structure].shape

In [ ]:
roi_pre = np.array([-2, 0])
roi_post = np.array([2, 2.5])

times_pre = ((roi_pre + 3) * 30000).astype(int)
times_post = ((roi_post + 3) * 30000).astype(int)

#### Theta band

In [ ]:
# window = 1500

# pre_icoh  = np.zeros((32,32))
# post_icoh = np.zeros((32,32))

data = lgamma_WT
for mouse in range(data.shape[2]):
    
    if mouse == 0:
        pre_icoh  = cm.icoh(data[:, 2000:3000, mouse], average = False)
        post_icoh  = cm.icoh(data[:, 3000:5000, mouse], average = False)
        
    else:
        pre_icoh  = np.dstack((pre_icoh, cm.icoh(data[:, 2000:3000, mouse], average = False)))
        post_icoh = np.dstack((post_icoh, cm.icoh(data[:, 3000:5000, mouse], average = False)))

    iteration += 1

pre_icoh_WT  = np.mean(pre_icoh, axis=2)
post_icoh_WT = np.mean(post_icoh, axis=2)

matrix_WT = post_icoh_WT
matrix_WT[1,0] = pre_icoh_WT[1,0]
matrix_WT[2,0] = pre_icoh_WT[2,0]
matrix_WT[3,0] = pre_icoh_WT[3,0]
matrix_WT[2,1] = pre_icoh_WT[2,1]
matrix_WT[3,1] = pre_icoh_WT[3,1]
matrix_WT[3,2] = pre_icoh_WT[3,2]


data = lgamma_KO
for mouse in range(data.shape[2]):
    
    if mouse == 0:
        pre_icoh  = cm.icoh(data[:, 2000:3000, mouse], average = False)
        post_icoh  = cm.icoh(data[:, 3000:5000, mouse], average = False)
        
    else:
        pre_icoh  = np.dstack((pre_icoh, cm.icoh(data[:, 2000:3000, mouse], average = False)))
        post_icoh = np.dstack((post_icoh, cm.icoh(data[:, 3000:5000, mouse], average = False)))

    iteration += 1

pre_icoh_KO  = np.mean(pre_icoh, axis=2)
post_icoh_KO = np.mean(post_icoh, axis=2)

matrix_KO = post_icoh_KO
matrix_KO[1,0] = pre_icoh_KO[1,0]
matrix_KO[2,0] = pre_icoh_KO[2,0]
matrix_KO[3,0] = pre_icoh_KO[3,0]
matrix_KO[2,1] = pre_icoh_KO[2,1]
matrix_KO[3,1] = pre_icoh_KO[3,1]
matrix_KO[3,2] = pre_icoh_KO[3,2]

In [ ]:
plt.figure(figsize=(150,150))

plt.matshow(matrix_WT)
plt.plot([0,1,2,3], color='white')
plt.xticks([0,1,2,3], ['mPFC', 'NAC', 'BLA', 'vHip'], rotation=0, fontsize=14)
plt.yticks([0,1,2,3], ['mPFC', 'NAC', 'BLA', 'vHip'], rotation=0, fontsize=14)
plt.text(0.1,2.3,'PRE', fontsize=16)
plt.text(2,1,'POST', fontsize=16)
plt.colorbar()
#plt.savefig(allFigs_dir + 'lgamma_synchrony_WT.pdf', dpi=150, format='pdf')



plt.figure(figsize=(150,150))

plt.matshow(matrix_KO)
plt.plot([0,1,2,3], color='white')
plt.xticks([0,1,2,3], ['mPFC', 'NAC', 'BLA', 'vHip'], rotation=0, fontsize=14)
plt.yticks([0,1,2,3], ['mPFC', 'NAC', 'BLA', 'vHip'], rotation=0, fontsize=14)
plt.text(0.1,2.3,'PRE', fontsize=16)
plt.text(2,1,'POST', fontsize=16)
plt.colorbar()
#plt.savefig(allFigs_dir + 'lgamma_synchrony_KO.pdf', dpi=150, format='pdf')


### Phase lag index

In [ ]:
data = theta_WT
z = signal.hilbert(data)
phases = np.angle(z)
#z_NAC = signal.hilbert(NAC)
#z_BLA = signal.hilbert(BLA)
#z_vHip = signal.hilbert(vHip)

In [ ]:
phases.shape

In [ ]:
window = 1500

# pre_icoh  = np.zeros((32,32))
# post_icoh = np.zeros((32,32))

data = phases
for mouse in range(data.shape[2]):
    
    if mouse == 0:
        pre_wpli  = cm.PLI2(data[:, 2000:3000, mouse], average = False)
        post_wpli  = cm.PLI2(data[:, 3000:5000+window, mouse], average = False)
        
    else:
        pre_wpli  = np.dstack((pre_wpli, cm.PLI2(data[:, 2000:3000, mouse], average = False)))
        post_wpli = np.dstack((post_wpli, cm.PLI2(data[:, 3000:5000, mouse], average = False)))
        

In [ ]:
pre_wpli.shape

In [ ]:
pre_wpli_all  = np.mean(pre_wpli, axis=2)
post_wpli_all = np.mean(post_wpli, axis=2)

In [ ]:
plt.figure(figsize=(150,150))
plt.matshow(pre_wpli_all)
#plt.xticks(ticks=np.arange(0, 32, 8), labels=['OFC', 'mPFC', 'DMS', 'BLA'], rotation=0)
#plt.yticks(ticks=np.arange(0, 32, 8), labels=['OFC', 'mPFC', 'DMS', 'BLA'], rotation=0)
plt.colorbar()
# plt.savefig(mydir + 'prePLI.png', dpi=150, format='png')

plt.figure(figsize=(150,150))
plt.matshow(post_wpli_all)
#plt.xticks(ticks=np.arange(0, 32, 8), labels=['OFC', 'mPFC', 'DMS', 'BLA'], rotation=0)
#plt.yticks(ticks=np.arange(0, 32, 8), labels=['OFC', 'mPFC', 'DMS', 'BLA'], rotation=0)
plt.colorbar()
# plt.savefig(mydir + 'postPLI.png', dpi=150, format='png')

###################################

#### Band-pass signal and plot power spectral density

In [ ]:
files = sorted(glob.glob("/home/maspe/filer/testMice/CC1973/HC_2018-04-24_16-56-27/*.continuous"))
channel = ps.loadContinuous(files[10])
data = channel['data']
sampleRate = channel['header']['sampleRate']
sampleRate

In [ ]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return b, a

N       = 2
fs      = sampleRate
lowcut  = 4.0
highcut = 12.0

b, a = butter_bandpass(lowcut, highcut, fs, order=N)

theta_band = signal.filtfilt(b=b, a=a, x=data - np.mean(data),
                           axis=-1, padtype='odd', padlen=None, method='pad', irlen=None)

In [ ]:
theta_band.shape

In [ ]:
fig = plt.figure()
plt.subplot(2,1,1)
plt.plot(data[10000:40000])
plt.subplot(2,1,2)
plt.plot(theta_band[10000:40000])

fig.savefig("/home/maspe/filer/testFiles/figs/theta.png", dpi=150, format='png')

In [ ]:
freqs = np.fft.fftfreq(len(data), sampleRate)

plt.plot(freqs[0:len(data)//2],2/len(data)*np.abs(theta_band[0:len(data)//2]))
plt.xlim([0,10])

In [ ]:
z = signal.hilbert(theta_band) #form the analytical signal
z

In [ ]:
inst_phase = np.unwrap(np.angle(z))
inst_phase

In [ ]:
plt.polar(z.real[:1000], z.imag[:1000], 'ro')

In [ ]:
f, psd = signal.welch(theta_band, fs=30000, window='hann', nperseg=None, noverlap=None, nfft=None,
                      detrend='constant', return_onesided=True, scaling='density', axis=-1, average='mean')

In [ ]:
# Plot the power spectrum
plt.figure(figsize=(11,3))
plt.semilogy(f,psd,'k')
# sns.despine()
plt.xlim((0,100))
plt.yticks(size=15)
plt.xticks(size=15)
plt.ylabel('power ($uV^{2}/Hz$)',size=15)
plt.xlabel('frequency (Hz)',size=15)
plt.title('PSD of Local Field Potential', size=20)
plt.show()

In [ ]:
# def butter_bandpass(lowcut, highcut, fs, order):
#     nyq = 0.5 * fs
#     low = lowcut / nyq
#     high = highcut / nyq
#     b, a = signal.butter(order, [low, high], btype='bandpass')
#     return b, a

# [b, a] = butter_bandpass(lowcut = 4, highcut = 8, fs = 30000.0, order = 3)